In [1]:
import warnings
warnings.filterwarnings('ignore')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import numpy as np
from nsepy import get_history
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime

In [3]:
import yfinance as yf
ticker='BELU.ME'
# Get the data for the stock AAPL
data = yf.download(ticker,'2018-01-01','2021-09-01')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-03,651.0,676.0,640.0,651.0,614.659546,90
2018-01-04,650.0,650.0,649.0,649.0,612.771118,3
2018-01-05,649.0,649.0,649.0,649.0,612.771118,0
2018-01-09,639.0,639.0,601.0,635.0,599.552673,20
2018-01-10,635.0,636.0,620.0,633.0,597.664307,750
...,...,...,...,...,...,...
2021-08-25,3334.0,3334.0,3147.0,3240.0,3240.000000,39496
2021-08-26,3274.0,3450.0,3200.0,3330.0,3330.000000,83258
2021-08-27,3340.0,3415.0,3307.0,3335.0,3335.000000,27278


In [4]:
df=pd.DataFrame(data['Adj Close'],index=data.index)
df=df.rename(columns={"Adj Close":"close"})

In [95]:
df_log

NameError: name 'df_log' is not defined

In [5]:
from dateutil.relativedelta import relativedelta
date_list = [datetime.datetime.strptime("2021-08-01", "%Y-%m-%d") + 
             relativedelta(days=x) for x in range(0,210)]


In [6]:
df.shape

(905, 1)

In [7]:
model_autoARIMA = auto_arima(df, start_p=0, start_q=0,
    test='adf',       # use adftest to find optimal 'd'
    max_p=10, max_q=10, # maximum p and q
                 # frequency of series
    d=None,           # let model determine 'd'
    seasonal=False,   # No Seasonality
    start_P=0, 
    max_P=10,
    start_Q=0,
    max_Q=10,
    D=2, trend='ct',
    
   stationary=False,                          
                             
    enforce_stationarity=True,
    trace=True,
    error_action='ignore',  
    suppress_warnings=True, 
    stepwise=True)
#print(model_autoARIMA.summary())
#print(model_autoARIMA.best_model())
print(model_autoARIMA)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=11381.249, Time=0.27 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=11365.889, Time=0.09 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=11356.273, Time=0.21 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=11381.249, Time=0.17 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=11347.318, Time=0.58 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=11303.861, Time=0.54 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=11328.395, Time=0.17 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=11300.429, Time=1.76 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=11327.186, Time=0.27 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=11299.887, Time=2.58 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=11318.894, Time=0.36 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=11302.679, Time=0.97 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=11303.130, Time=2.60 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=11302.010, Time=1.89 sec
 ARIMA(5,1,0)(0,0,0

In [8]:
model = ARIMA(df, order=(4, 1, 1))
fitted = model_autoARIMA.fit(df,disp=1)
print(fitted.summary())

C:\Users\ArtMed\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\ArtMed\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  905
Model:               SARIMAX(4, 1, 1)   Log Likelihood               -5641.944
Date:                Mon, 06 Sep 2021   AIC                          11299.887
Time:                        16:43:00   BIC                          11338.342
Sample:                             0   HQIC                         11314.574
                                - 905                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.7075     11.574     -0.061      0.951     -23.393      21.978
drift          0.0038      0.015      0.258      0.797      -0.025       0.033
ar.L1          0.9561      0.052     18.336      0.0

In [9]:
fc, se, conf = fitted.predict(future, alpha=0.05)  # 95% confidence

NameError: name 'future' is not defined

In [10]:
future = pd.DataFrame(index=date_list, columns= df.columns)
df2 = pd.concat([df, future])

In [11]:
fc, se, conf = fitted.predict(., alpha=0.05)  # 95% confidence

TypeError: n_periods must be an int

In [12]:
fc_series = pd.Series(fc, index=future.index)
lower_series = pd.Series(conf[:, 0], index=future.index)
upper_series = pd.Series(conf[:, 1], index=future.index)
plt.figure(figsize=(12,5), dpi=100)
plt.grid(True)
plt.plot(df, label='training')
plt.plot(future, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title(ticker+' Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Actual Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

NameError: name 'fc' is not defined

In [103]:
print('from {} to {}, predicted {}'.format(lower_series[-1], upper_series[-1], fc_series[-1]))

from 169.83330094530965 to 320.3881359145051, predicted 245.11071842990737


In [104]:
fc_series['2021-09-02']

222.35610120494883

In [105]:
lower_series['2021-09-02']

192.44149993482446

In [106]:
upper_series['2021-09-02']

252.2707024750732

In [46]:
(160-153)/160

0.04375